# Projet - Modélisation et simulation d'un système de vélos partagés - SPEISMANN & POURNY

In [128]:
import numpy as np
import random as rd
from math import sqrt

## Calibration des paramètres

### Données

In [4]:
# Matrice du temps de trajet moyen entre les stations:
Tau = np.array([[0, 3, 5, 7, 7],
                [2, 0, 2, 5, 5],
                [4, 2, 0, 3, 3],
                [8, 6, 4, 0, 2],
                [7, 7, 5, 2, 0]])

# Matrice de routage (Proba de passage d'un vélo de i vers j):
P = np.array([[0, 0.22, 0.32, 0.20, 0.26],
              [0.17, 0, 0.34, 0.21, 0.28],
              [0.19, 0.26, 0, 0.24, 0.31],
              [0.17, 0.22, 0.33, 0, 0.28],
              [0.18, 0.24, 0.35, 0.23, 0]]) 

# Intensité de départ par heure par station: 
Lambda = np.array([2.8, 3.7, 5.5, 3.5, 4.6])

# Nombre de vélos initial dans chaque station:
V0 = np.array([20, 15, 17, 13, 18])

### Définir les Paramètres

Seul les $\mu$ se calculent à partir des données.
 
Les $\lambda$ et les $p_{i,j}$ sont automatiquement déductible des données 

In [10]:
Mu = [[None if i == j else 1 / Tau[i, j] for j in range(5)] for i in range(5)]

[[None, 0.3333333333333333, 0.2, 0.14285714285714285, 0.14285714285714285],
 [0.5, None, 0.5, 0.2, 0.2],
 [0.25, 0.5, None, 0.3333333333333333, 0.3333333333333333],
 [0.125, 0.16666666666666666, 0.25, None, 0.5],
 [0.14285714285714285, 0.14285714285714285, 0.2, 0.5, None]]

## Simulation

### Définir l'état initial:

In [ ]:
# Etat inital:
S0 = np.concatenate([V0, np.zeros(5 * 4, dtype=int)])  # On suppose que le système commence sans vélo en trajet.

Tableau d'aide pour récupérer les indices des $S_{T_{i,j}}

In [107]:
index_S_T = [[None, 5, 6, 7, 8],
        [9, None, 10, 11, 12],
        [13, 14, None, 15, 16],
        [17, 18, 19, None, 20],
        [21, 22, 23, 24, None]]

### Définir une simulation

In [137]:
def simulation(S = S0, T = 150):
    t = 0
    temps = [t]
    historique = [S.copy()]
    while t < T:
        taux = [[[None if i == j else Lambda[i] * P[i][j] * (S[i]>0) for j in range(5)] for i in range(5)], 
                [[None if i == j else Mu[i][j] * S[index_S_T[i][j]] for j in range(5)] for i in range(5)]]
        taux_sortie_etat = sum([taux[0][i][j] + taux[1][i][j] for i in range(5) for j in range(5) if i != j])

        t+=np.random.exponential(1/taux_sortie_etat) # Le taux est bien la somme, spécificité de la fonction.
        temps.append(t)
        # Evenement qui a lieu:
        indices = [(k, i, j) for k in range(2) for i in range(5) for j in range(5) if i != j]
        poids = np.array([taux[k][i][j] for k, i, j in indices], dtype=float)
        prob = poids / taux_sortie_etat
        k_evt, i_evt, j_evt = indices[np.random.choice(len(indices), p=prob)]

        # Départ d'un vélo de i vers j:
        if k_evt == 0:
            S[i_evt] -= 1
            S[index_S_T[i_evt][j_evt]] += 1
            
        # Arrivé d'un vélo de i vers j:
        elif k_evt == 1:
            S[index_S_T[i_evt][j_evt]] -= 1
            S[j_evt] += 1
            
        historique.append(S.copy())
    return S, historique, temps

### Estimation d'une station vide à l'horizon

In [146]:
M = 200
res = 0
historique = [S0]
for i in range(M):
    print("Simulation numéro: ", i+1)
    S, _, _ = simulation()
    if 0 in S[0:5]:
        res += 1
    
p = res/M
print(p)

Simulation numéro:  1
Simulation numéro:  2
Simulation numéro:  3
Simulation numéro:  4
Simulation numéro:  5
Simulation numéro:  6
Simulation numéro:  7
Simulation numéro:  8
Simulation numéro:  9
Simulation numéro:  10
Simulation numéro:  11
Simulation numéro:  12
Simulation numéro:  13
Simulation numéro:  14
Simulation numéro:  15
Simulation numéro:  16
Simulation numéro:  17
Simulation numéro:  18
Simulation numéro:  19
Simulation numéro:  20
Simulation numéro:  21
Simulation numéro:  22
Simulation numéro:  23
Simulation numéro:  24
Simulation numéro:  25
Simulation numéro:  26
Simulation numéro:  27
Simulation numéro:  28
Simulation numéro:  29
Simulation numéro:  30
Simulation numéro:  31
Simulation numéro:  32
Simulation numéro:  33
Simulation numéro:  34
Simulation numéro:  35
Simulation numéro:  36
Simulation numéro:  37
Simulation numéro:  38
Simulation numéro:  39
Simulation numéro:  40
Simulation numéro:  41
Simulation numéro:  42
Simulation numéro:  43
Simulation numéro:  

#### Calcul de l'intervalle de confiance (à 95%)

In [147]:
incertitude = 1.96 * sqrt(p * (1-p) / M)
print([p - incertitude, p + incertitude])

[0.6738343380645644, 0.7961656619354356]


### Estimation du temps où au moins une station est vide sur une simulation

In [148]:
_, historique, temps = simulation()
N = len(temps)
res = 0
for i in range(N-1):
    if 0 in historique[i][0:5]:
        res += temps[i+1] - temps[i]

print(res/temps[-1])

0.747518405386027
